<a href="https://colab.research.google.com/github/financieras/pyCourse/blob/main/ejemplos/caso3_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reto 1
Estudie los archivos:

* calisto3_0260.ipynb VAN y TIR
* calisto3_0270.ipynb Préstamos
* calisto3_0280.ipynb Bonos, ETTI, forward, arbitraje